Import the necessary imports

In [ ]:
from __future__ import print_function, division, absolute_import

import tensorflow as tf
from tensorflow.contrib import keras

import numpy as np
import os
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
import itertools
import cPickle
import h5py
from matplotlib import pyplot as plt
%matplotlib inline

Now read the data

In [ ]:
with h5py.File("11S_LP_DS.h5", "r") as hf:
  LFP_features_train = hf["LFP_features_train"][...]
  targets_train = hf["targets_train"][...]
  speeds_train = hf["speeds_train"][...]
  LFP_features_eval = hf["LFP_features_eval"][...]
  targets_eval = hf["targets_eval"][...]
  speeds_eval = hf["speeds_eval"][...]

And make sure it looks ok

In [ ]:
rand_sample = np.random.randint(LFP_features_train.shape[0])
for i in range(LFP_features_train.shape[-1]):
  plt.figure(figsize=(20,7))
  plt_data = LFP_features_eval[93600,:,i]
  plt.plot(np.arange(-0.5, 0., 0.5/plt_data.shape[0]), plt_data)
  plt.xlable("time")
  plt.title(str(i))

Now we write some helper functions to easily select regions.

In [ ]:
good_channels = np.load('good_channels.npy')
front_idxs = np.where([good_channels<64])[0]
back_idxs = np.where([good_channels>=64])[0]
bottom_channels = [76,83,80,78,89,100,70,64,94,61,67,42,84,52,56,38,22,8,24,39,4,26,47,
                           37,45,59,46,25,7]
bottom_idxs = np.sort(np.stack([np.where(good_channels==bc)[0] for bc in bottom_channels]).squeeze())
top_idxs = np.stack([i for i in range(good_channels.shape[0]) if i not in bottom_idxs]).squeeze()

In [ ]:
LFP_features_train_current = LFP_features_train
LFP_features_eval_current = LFP_features_eval
# LFP_features_train_current = LFP_features_train[...,bottom_idxs]
# LFP_features_eval_current = LFP_features_eval[...,bottom_idxs]

Create a call back to save the best validation accuracy

In [ ]:
model_chk_path = 'my_model.hdf5'
mcp = keras.callbacks.ModelCheckpoint(model_chk_path, monitor="val_acc",
                      save_best_only=True)

Below I have defined a couple of different network architectures to play with.

In [ ]:
# try:
#   model = None
# except NameError:
#   pass
# decay = 1e-3
# conv1d = keras.layers.Convolution1D
# maxPool = keras.layers.MaxPool1D
# model = keras.models.Sequential()
# model.add(conv1d(64, 5, padding='same', strides=2, activation='relu', 
#                  kernel_regularizer=keras.regularizers.l2(decay),
#                  input_shape=LFP_features_train_current.shape[1:]))
# model.add(maxPool())
# model.add(conv1d(128, 3, padding='same', strides=2, activation='relu', 
#                  kernel_regularizer=keras.regularizers.l2(decay)))
# model.add(maxPool())
# model.add(conv1d(128, 3, padding='same', strides=2, activation='relu', 
#                  kernel_regularizer=keras.regularizers.l2(decay)))
# model.add(maxPool())
# model.add(conv1d(128, 3, padding='same', strides=2, activation='relu', 
#                  kernel_regularizer=keras.regularizers.l2(decay)))
# model.add(maxPool())
# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dropout(rate=0.5))
# model.add(keras.layers.Dense(2, activation='softmax', kernel_regularizer=keras.regularizers.l2(decay)))

In [ ]:
# try:
#   model = None
# except NameError:
#   pass
# decay = 1e-3
# conv1d = keras.layers.Convolution1D
# maxPool = keras.layers.MaxPool1D
# model = keras.models.Sequential()
# model.add(conv1d(33, 5, padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(decay),
#                  input_shape=LFP_features_train.shape[1:]))
# model.add(maxPool())
# model.add(conv1d(33, 3, padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(decay)))
# model.add(maxPool())
# model.add(conv1d(16, 3, padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(decay)))
# model.add(maxPool())
# model.add(conv1d(4, 3, padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(decay)))
# model.add(maxPool())
# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dropout(rate=0.5))
# model.add(keras.layers.Dense(2, activation='softmax', kernel_regularizer=keras.regularizers.l2(decay)))

In [ ]:
try:
  model = None
except NameError:
  pass
decay = 1e-3
regul = keras.regularizers.l1(decay)
conv1d = keras.layers.Convolution1D
maxPool = keras.layers.MaxPool1D
BN = keras.layers.BatchNormalization
Act = keras.layers.Activation('relu')
model = keras.models.Sequential()
model.add(keras.layers.Convolution1D(64, 5, padding='same', strides=2, 
                 kernel_regularizer=keras.regularizers.l1(decay),
                 input_shape=LFP_features_train_current.shape[1:]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool1D())
model.add(keras.layers.Convolution1D(128, 3, padding='same', strides=2,
                 kernel_regularizer=keras.regularizers.l1(decay)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool1D())
model.add(keras.layers.Convolution1D(128, 3, padding='same', strides=2,
                 kernel_regularizer=keras.regularizers.l1(decay)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))

# model.add(keras.layers.GlobalMaxPooling1D())
model.add(keras.layers.MaxPool1D())
model.add(keras.layers.Convolution1D(128, 3, padding='same', strides=2,
                 kernel_regularizer=keras.regularizers.l1(decay)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
# model.add(maxPool())
# model.add(keras.layers.Flatten())
model.add(keras.layers.GlobalMaxPooling1D())
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(2, activation='softmax', kernel_regularizer=keras.regularizers.l1_l2(decay)))

In [ ]:
# try:
#   model = None
# except NameError:
#   pass
# decay = 1e-3
# conv1d = keras.layers.Convolution1D
# maxPool = keras.layers.MaxPool1D
# model = keras.models.Sequential()
# model.add(conv1d(64, 5, padding='same', strides=2, activation='relu', 
#                  kernel_regularizer=keras.regularizers.l2(decay),
#                  input_shape=LFP_features_train_current.shape[1:]))
# model.add(maxPool())

# model.add(conv1d(128, 3, padding='same', strides=1, activation='relu', 
#                  kernel_regularizer=keras.regularizers.l2(decay)))
# model.add(maxPool())

# model.add(keras.layers.LSTM(128))
# # model.add(conv1d(128, 3, padding='same', strides=2, activation='relu', 
# #                  kernel_regularizer=keras.regularizers.l2(decay)))
# # model.add(maxPool())
# # model.add(conv1d(128, 3, padding='same', strides=2, activation='relu', 
# #                  kernel_regularizer=keras.regularizers.l2(decay)))
# # model.add(maxPool())
# # model.add(layers.Dense(1, activation='sigmoid'))
# model.add(keras.layers.Dense(2, activation='softmax', kernel_regularizer=keras.regularizers.l2(decay)))

In [ ]:
# try:
#   model = None
# except NameError:
#   pass
# decay = 1e-3
# regul = keras.regularizers.l1_l2(decay)
# conv1d = keras.layers.Convolution1D
# maxPool = keras.layers.MaxPool1D
# BN = keras.layers.BatchNormalization
# Act = keras.layers.Activation('relu')
# model = keras.models.Sequential()
# model.add(conv1d(64, 5, padding='same', strides=2, 
#                  kernel_regularizer=regul,
#                  input_shape=LFP_features_train_current.shape[1:]))
# model.add(BN())
# model.add(Act)
# model.add(maxPool())
# model.add(conv1d(128, 3, padding='same', strides=2,
#                  kernel_regularizer=regul))
# model.add(BN())
# model.add(Act)
# model.add(maxPool())
# model.add(keras.layers.LSTM(128, dropout=0.1, recurrent_dropout=0.5))
# # model.add(conv1d(128, 3, padding='same', strides=2, activation='relu', 
# #                  kernel_regularizer=keras.regularizers.l2(decay)))
# # model.add(maxPool())
# # model.add(conv1d(128, 3, padding='same', strides=2, activation='relu', 
# #                  kernel_regularizer=keras.regularizers.l2(decay)))
# # model.add(maxPool())
# # model.add(layers.Dense(1, activation='sigmoid'))
# model.add(keras.layers.Dense(2, activation='softmax', kernel_regularizer=keras.regularizers.l2(decay)))

In [ ]:
model.compile(optimizer='Adam',
loss='categorical_crossentropy',
metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(LFP_features_train_current,
                    targets_train,
                    epochs=20,
                    batch_size=1024,
                    validation_data=(LFP_features_eval_current, targets_eval),
                    callbacks=[mcp])

Helper function for the confusion matrix

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting `normalize=True`.
  """
  if normalize:
      cm = cm.astype('float') / np.maximum(cm.sum(axis=1)[:, np.newaxis],1.0)
      print("Normalized confusion matrix")
  else:
      print('Confusion matrix, without normalization')

  print(cm)
  
  cm = (cm*1000).astype(np.int16)
  cm = np.multiply(cm, 0.1)
  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, "{0}%".format(cm[i, j]),
               horizontalalignment="center",
               color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return plt.gcf()

In [ ]:
class_names = ['go', 'stop']
model.load_weights('my_model_bottom.hdf5')
y_pred_initial = model.predict(LFP_features_eval)
targets_eval_1d = np.argmax(targets_eval, axis=1)
y_pred = np.argmax(y_pred_initial, axis=1)
cnf_matrix = confusion_matrix(targets_eval_1d, y_pred)
np.set_printoptions(precision=2)
plt.figure()
fig = plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

In [ ]:
wrong_idxs = np.where(y_pred != targets_eval_1d)[0]
wrong_vals = speeds_eval[wrong_idxs]
# wrong_vals.squeeze().shape
# crazy_wrong_idxs.shape

In [ ]:
plt.cla()
plt.close()
plt.figure(figsize=(20,7))
n, bins, patches = plt.hist(wrong_vals.squeeze(), 
                            bins=np.arange(0,1,0.01),)

plt.plot(bins)
plt.xlim([0,1])
fig_dist = plt.gcf()

Train and evaluation accuracies

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.figure(figsize=(20,7))
plt.plot(epochs, acc, 'bo', label='Training')
plt.plot(epochs, val_acc, 'b', label='Validation')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right', fontsize=24)
plt.xticks(np.arange(20))

As a sanity check, we make sure the mean impedance of channels in different regions are not that different

In [ ]:
impedances = np.genfromtxt('/home/atabak/test_day/IMPEDANCE.csv', delimiter=',', usecols=4)[1:]
bottom_channels = [76,83,80,78,89,100,70,64,94,61,67,42,84,52,56,38,22,8,24,39,4,26,47,
                           37,45,59,46,25,7]
top_channels = [i for i in good_channels if i not in bottom_channels]

front_channels = good_channels[good_channels<64]
back_channels = good_channels[good_channels>=64]
print("mean of bottom good channels is: {0}".format(impedances[bottom_channels].mean()))
print("mean of top good channels is: {0}".format(impedances[top_channels].mean()))
print("mean of front good channels is: {0}".format(impedances[front_channels].mean()))
print("mean of back good channels is: {0}".format(impedances[back_channels].mean()))
print("mean of good channels is: {0}".format(impedances[good_channels].mean()))